In [2]:
from torchPIV.PIVbackend import ToTensor, PIVDataset, moving_window_array
from SIV_library.lib import block_match, moving_reference_array, match_to_displacement, correlation_to_displacement, get_field_shape, get_x_y, plot_velocity_single_frame
from SIV_library.processing import Video, Processor, Viewer

import os
import sys
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

from collections.abc import Collection


os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")
import helpers


%load_ext autoreload
%autoreload 2

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

dataset = PIVDataset("../Test Data/plume simulation_PROCESSED", ".jpg",
                     "sequential", transform=ToTensor(dtype=torch.uint8))

In [4]:
# testing_0 = helpers.imread_normalized_float_grayscale(os.path.join("../Test Data/test", "0.jpg"))
# testing_0 = np.where(testing_0 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
# testing_1 = helpers.imread_normalized_float_grayscale(os.path.join("../Test Data/test", "1.jpg"))
# testing_1 = np.where(testing_1 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL

In [11]:
img_a, img_b = dataset[1]
img_a, img_b = img_a.to(device), img_b.to(device)

# img_a, img_b = torch.from_numpy(testing_0), torch.from_numpy(testing_1)
window_size, overlap = 64, 32 
aa = moving_window_array(img_a, window_size, overlap)

window_amount = aa.shape[0]
print(window_amount, 'windows')

3969 windows


In [ ]:
bb = moving_reference_array(img_b, window_size, overlap, left=40, right=40, top=40, bottom=40)

idx = 2300
window, area = aa, bb
# idx = 5

#4.09458182656812
#-3.4124938088181693

cv2.imshow('Interrogation window', window[idx].cpu().numpy())
cv2.waitKey(0)
cv2.imshow('Search area', area[idx].cpu().numpy())
cv2.waitKey(0)
cv2.destroyAllWindows()
# corr = block_match(window, area, mode=0)[idx]
# intensity = block_match(window, area, mode=1)[idx]

# corr_min, corr_max = torch.min(corr), torch.max(corr)
# corr_img = (corr-corr_min)/(corr_max-corr_min)
# 
# intensity_min, intensity_max = torch.min(intensity), torch.max(intensity)
# intensity_img = 1 - (intensity-intensity_min)/(intensity_max-intensity_min)

# match_to_displacement(block_match(window, area, 1))

corr = block_match(window, area, mode=1)
nrows, ncols = get_field_shape(img_a.shape, window_size, overlap)
u, v = correlation_to_displacement(corr, nrows, ncols)
# row, col = idx // 31, idx % 31
# print(f"u shape = {u.shape}")
# print(u[row, col])
# print(f"v shape = {v.shape}")
# print(v[row, col])

# plts = np.hstack((corr_img.cpu().numpy(), intensity_img.cpu().numpy()))

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
x, y = np.meshgrid(range(area.shape[2] - window.shape[2] + 1), range(area.shape[1] - window.shape[1] + 1))


# ax.plot_surface(x, y, corr_img, color='b', alpha=0.5, label='correlation')
# ax.plot_surface(x, y, intensity_img, color='r', alpha=0.5, label='SAD')

# plt.legend()
# plt.show()


 73%|███████▎  | 59/81 [04:46<01:37,  4.41s/it]

In [9]:
x_c, y_c = get_x_y(img_a.shape, window_size, overlap)   

In [10]:
plot_velocity_single_frame(img_a, x_c, y_c, u, v)

In [77]:
def vector_field(filename: str, results: np.ndarray, scale: float) -> None:
    fig, ax = plt.subplots()

    velocities = results[:, 2:]  # exclude reference frame (only nan/zero values)
    abs_velocities = np.sqrt(velocities[:, 0]**2 + velocities[:, 1]**2)
    min_abs, max_abs = np.min(abs_velocities, axis=0), np.max(abs_velocities, axis=0)

    frame = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    image = ax.imshow(frame, cmap='gray')

    # Note: velocity scaling is done for correct color mapping and quiver length
    x0, y0, vx0, vy0 = results[0]
    new_x0, new_y0 = x0/scale, y0/scale
    # vx0, vy0 = np.flip(vx0, axis=0), np.flip(vy0, axis=0)  # flip velocities for correct IMAGE coords
    vectors = ax.quiver(new_x0, new_y0, vx0, vy0, max_abs-min_abs, cmap='jet')
    plt.show()

In [78]:
video_file = "plume simulation.mp4"
image_1_file = rf"../Test Data/plume simulation_PROCESSED/00101.jpg"
fn = video_file.split(".")[0]
frames = [int("00" + str(i)) for i in range(100, 151)]

vid = Video(rf"../Test Data/{video_file}", df='.jpg', indices=frames)
vid.create_frames()
vid.create_frames()

processor = Processor(rf"../Test Data/{fn}", df='.jpg', denoise=False, rescale=None, crop=False)
processor.postprocess()

device = torch.cuda.get_device_name() if torch.cuda.is_available() else "cpu"
print(device)

capture_fps = 240.
scale = 1.

viewer = Viewer(rf"../Test Data/{fn}_PROCESSED", playback_fps=30., capture_fps=capture_fps)

shape = int(np.sqrt(y_c.shape[0]))
# viewer.play_video()
# res = np.array((x_c.reshape((shape, shape)), y_c.reshape((shape, shape)), u, v))
res = np.array([[x_c.reshape(shape, shape), y_c.reshape(shape, shape), u, v]])
print(res.shape)
vector_field(image_1_file, res, scale)

Directory '../Test Data/plume simulation' already exists
Directory '../Test Data/plume simulation' already exists
Directory '../Test Data/plume simulation_PROCESSED' already exists
cpu
(1, 4, 63, 63)
